# pyBOS demonstration notebook



## imports
This notebook is not intented to be a lecture about BOS but only a demonstration
of what can be done with the library pyBOS.

First we import packages that we'll use later.
`%matplotlib widget` is here for improving plots quality in this notebook

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
from pathlib import Path

%matplotlib widget

Then we import stuff fron the pyBOS library

In [ ]:
from pybos.reader import reader, DANTEC_DAT
from pybos.vector_calculus import divergence, amplitude
from pybos.laplacian import solve_BOS
from pybos.extraction import idz2n
from pybos.physical_laws import gladstone_dale, ideal_gaz

## Reading the data

We use the `reader` function to read the data.
It must be a `.csv` type file (including dome of the tecplot's `.plt.` files)

If possible prefer using the calibration from Dantec/Davis and avoid importing data in pixel.

In [ ]:
im_datafile = Path("data/example_data_axi.vect")
cols = 0, 1, 2, 3  # pix for now
scale = 1 / 77.0  # pix to mm

data = reader(
    im_datafile, DANTEC_DAT, usecols=cols, scale=scale, unit="mm", output_unit="mm", skipfooter=0
)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))
data.U.plot.pcolormesh(ax=ax1, cmap=plt.cm.jet)
data.V.plot.pcolormesh(ax=ax2, cmap=plt.cm.jet)
ax1.axis("equal")
ax2.axis("equal")
fig.tight_layout()
plt.show()

# data

## Preparing the data

You may want to crop the data to remove reflection and other defects.
Here we are only using the `isel` method of `Xarray`'s `Dataset`.

In [ ]:
roi = {"x": slice(None, None), "y": slice(20, -20)}

cropped_data = data.isel(roi)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))
data.U.plot.pcolormesh(ax=ax1, cmap=plt.cm.jet)
(cropped_data.U * 0.0).plot.pcolormesh(ax=ax1, alpha=0.5, add_colorbar=False)
data.V.plot.pcolormesh(ax=ax2, cmap=plt.cm.jet)
(cropped_data.V * 0.0).plot.pcolormesh(ax=ax2, alpha=0.5, add_colorbar=False)
ax1.axis("equal")
ax2.axis("equal")
fig.tight_layout()
plt.show()

cropped_data

You may also want to filter the data. Here we are only using standard filter from `scipy`.

In order to keep the `Xarray`'s `Dataset` data format, we use the `apply_ufunc` wrapper.

In [ ]:
filt_method = "spline"
if filt_method == "gaussian":
    from scipy.ndimage import gaussian_filter

    sigma = 2, 2
    filter_ = lambda x: gaussian_filter(x, sigma=sigma, order=0, mode="nearest")

elif filt_method == "spline":
    from scipy.signal import spline_filter

    lmbda = 2
    filter_ = lambda x: spline_filter(x, lmbda=lmbda)

filtered_data = xr.apply_ufunc(filter_, cropped_data, keep_attrs=True)

filtered_data

There is a function for computing the displacement's amplitude and divergence.

In [ ]:
filtered_data["amplitude"] = amplitude(filtered_data)
filtered_data["divergence"] = divergence(filtered_data)

print(f"mean(amplitude)={float(filtered_data.amplitude.mean())}")
print(f"mean(divergence)={float(filtered_data.divergence.mean())}")

fig, axs = plt.subplots(2, 2, figsize=(9, 9))
filtered_data.U.plot.pcolormesh(ax=axs[0][0], cmap=plt.cm.jet)
filtered_data.V.plot.pcolormesh(ax=axs[0][1], cmap=plt.cm.jet)
filtered_data.amplitude.plot.pcolormesh(ax=axs[1][0], cmap=plt.cm.jet)
filtered_data.divergence.plot.pcolormesh(ax=axs[1][1], cmap=plt.cm.jet)

for axx in axs:
    for axy in axx:
        axy.axis("equal")
fig.tight_layout()
plt.show()

## Resolution

Now we define the properties of the experiment (size of the tank, boundary conditions)
and the physical law for converting refracting index to to temperature and conversely.

In [ ]:
d = 55  # [mm] object-pattern distance
w = float(filtered_data.x.max() - filtered_data.x.min())  # [mm] object size
n0 = 1.00027  # [] refractive index of the air

idz_top = idz_bottom = None
idz_left = n0 * w
idz_right = n0 * w

And compute the refractive index

In [ ]:
idz = solve_BOS(filtered_data.divergence, w, d, n0, idz_top, idz_bottom, idz_left, idz_right)

fig, ax = plt.subplots()
idz.plot.pcolormesh(ax=ax, cmap=plt.cm.jet)
ax.axis("equal")
fig.tight_layout()
plt.show()

idz

$n$ can be extracted using an axisymmetry hypothesis

In [ ]:
n_abel = idz2n(idz, w, n0, method="abel")
n_radon = idz2n(idz, w, n0, method="radon")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
n_radon.plot.pcolormesh(ax=ax1, cmap=plt.cm.jet)
ax1.set_title("with Radon")
ax1.axis("equal")
n_abel.plot.pcolormesh(ax=ax2, cmap=plt.cm.jet)
ax2.set_title("with Abel")
ax2.axis("equal")
fig.tight_layout()
plt.show()

n_abel

That we can convert to density using Gladstone–Dale relation

In [ ]:
G = 0.000229  # [m³/Kg] Gladstone constant of air at 20°C and 1 bar

density = gladstone_dale(n_abel, G)

fig, ax = plt.subplots()
density.plot.pcolormesh(ax=ax, cmap=plt.cm.jet)
ax.axis("equal")
fig.tight_layout()
plt.show()

That we can convert to temperature using the Ideal Gaz law

In [ ]:
p = 101_325  # [Pa] = [J/m³] pressure
M = 28.976 * 1e-3  # [kg/mol] molar mass

T = ideal_gaz(density, p, M)

fig, ax = plt.subplots()
T.plot.pcolormesh(ax=ax, cmap=plt.cm.jet)
ax.axis("equal")
fig.tight_layout()
plt.savefig("Temperature_axi.png", dpi=200)

T

Now it is your turn to use the library ;) !